In [1]:
## Not great, has names

with open('words.txt', 'r') as f:
    allwords = f.read().split()

In [ ]:
import numpy as np

ALLCAPS = {l:u for (l,u) in zip('abcdefghijklmnopqrstuvwxyz', 'ABCDEFGHIJKLMNOPQRSTUVWXYZ')}

## BASIC HELPER FUNC
def chunks(w):
    return [e for e in w]

def compare(w1, w2):
    """First round of comparing: positional comparison"""
    return [a==b for (a,b) in zip(w1, w2)]

def count_c(c, w):
    """counts characters in w (can be list or str)"""
    return len([e for e in w if e==c])

def has_valid_char(word, char_bank):
    """Check if word has chars found in char_bank"""
    return len(set(word) - char_bank) == 0

def is_unordered_sublist(sub, super_):
    """check if sub is contained in super_ (both iterables)"""
    bools = []
    for c in set(sub):
        if c in super_ and count_c(c, sub) <= count_c(c, super_): # <= in case the priority_char added less than the actual word
            bools.append(1)
        else:
            bools.append(0)
            
    return 0 not in bools

## INFO THEORY
def entropy(word):
    """Shannon Entropy of a string"""
    prob_log_prob = []
    for c in set(word):
        p = count_c(c, word)/len(word)
        prob_log_prob.append(p*np.log(p))
    return -sum(prob_log_prob)

# algorithm

1. start with n-letter word with highest entropy
Conditions:

Given letter bank: if a word doesn't have a letter in the letter bank, reject

Given prioirty letters: if a word doesn't have one, reject

Given fixed positions: if a word doesn't have letters there, reject

Data science questions:
- are there any words for which it is very hard for this algorithm (or human) to guess within 6 tries?

In [100]:
class Wordle:
    """class to simulate Wordle"""
    def __init__(self, word):
        self.word = word
        self.length = len(word)
        self.history = [] # unused outside class
        self.solved = False
    
    def show_flags(self, guess, flags):
        """print guess along with its flags side by side"""
        for c in guess:
                print(ALLCAPS[c], '', end='')
        print()
        for f in flags:
            print(f, '', end='')
    
    def verify(self, guess):
        """Verify if guess is correct, and if not gives clues (Green, Yellow, Gray)"""
        # include try/catch to make sure green letters are there? (not in game)
        flags = []
        c_encounter = {c:count_c(c, self.word) for c in set(self.word)}
        if guess == self.word:
            print('correct!')
            flags = ['+' for i in range(self.length)]
            self.solved = True
            return
        word2word = compare(guess, self.word)
        for ix, b in enumerate(word2word):
            char = guess[ix]
            ## GREEN right character, right position
            if b:
                flags.append('+') #green
                c_encounter[char] -= 1
            else:
                ## YELLOW right character, wrong position
                ## accounts for character count. e.g., if character count of 1
                ## and shown yellow flag, should be gray on its next encounter in the guess word
                if char in self.word and c_encounter[char] > 0: 
                    flags.append('!') #yellow
                    c_encounter[char] -= 1
                ## GRAY no such character
                else:
                    flags.append('-')
        
        self.history.append(flags)
        self.show_flags(guess, flags)
        return flags
    
class WordleInference:
    """Contains features/inference data"""
    def __init__(self, length, word_bank, truth):
        self.optimal = ['_' for i in range(length)] # fixes green letters
        self.word_bank = word_bank
        self.priority_char = [] #accounts for repeated char (hence a list)
        self.char_bank = {c for c in 'abcdefghijklmnopqrstuvwxyz'}
#         self.null_opt =  ['_' for i in range(length)] # _____
        self.candidate_hist = dict()
        
        #debugging
        self.truth = truth #ground truth
        
    def update_priors(self, guess, flags):
        """ Update inferred word features based on flags
        theoretically like gradient descent
        """
        priority_seen = set(self.priority_char) #Yellow characters seen from previous guess
        for ix, (c, f) in enumerate(zip(guess, flags)):
            if f == '+': # Green
                self.optimal[ix] = c
            elif f == '!' and c not in priority_seen: # Yellow
                self.priority_char.append(c)
            elif f == '-' and c not in self.priority_char + self.optimal: # Gray, second boolean for edge case
                try:
                    self.char_bank.remove(c)
                except KeyError: # for duplicate absent letters
                    pass
        try:
            if guess == self.truth:
                print('update_priors: correct word removed')
            self.word_bank.remove(guess)
        except ValueError: #list.remove(x): x not in list TODO WHY IS THIS HAPPENING HERE
            pass

## DEBUGGING
## if statements with prints
    def next_guess(self, it=None): #it: debug iteration
        """Find most likely next guess based on updated priors in previous step"""
        candidates = []
        preferred_candidates = [] # for those with letters that match exactly
        print(f'{self.truth} in word bank: {self.truth in self.word_bank}')
#         self.temp_bank = []
        word_bank = self.word_bank
        for w in word_bank:
#             self.temp_bank.append(w)
            if w == self.truth:
                print(f'{self.truth} in loop!')
            ## skip words with gray letters
            if not has_valid_char(w, self.char_bank):
                if w == self.truth:
                    print('has_valid_char: correct word removed', sorted(self.char_bank))
                self.word_bank.remove(w)
                continue
            ## skip words without yellow letters
            if not is_unordered_sublist(self.priority_char, w):
                if w == self.truth:
                    print('is_unordered_sublist: correct word removed', self.priority_char)
                self.word_bank.remove(w)
                continue
            # compare these two variables to decide if candidate
            fixed_char_count = 0 #number of green letters
            w_sim_count = 0  # see if current word shares the above
            
            for ix, c in enumerate(self.optimal):
                if c != '_':
                    fixed_char_count += 1
                    if w[ix] == c:
                        
                        w_sim_count += 1
            if fixed_char_count == w_sim_count:
                candidates.append(w)
            else:

                if w == self.truth:
                    print('correct letters: correct word removed', self.optimal)
                
                self.word_bank.remove(w)
                if len(candidates) == 0: #worst case scenario
                    candidates.append(w)
                    self.word_bank.append(w)
                
                
        print(len(candidates), 'candidates left in iter | ')
#         print('word banks same', set(temp_bank)==set(self.word_bank))
        self.candidate_hist[it] = candidates
        if len(candidates) == 0:
            print(self.word_bank)
        return candidates[np.argmax([entropy(w) for w in candidates])] 

In [101]:
word = 'knoll'
word_bank = [w for w in allwords if len(w) == len(word)]
# np.random.shuffle(word_bank) # enables starting with random choice from all high entropy words
# guess = word_bank[np.argmax([entropy(w) for w in word_bank])] #start guess
# guess = 'baths'
guess = 'paced'
wordle = Wordle(word)
inferer = WordleInference(len(word), word_bank, word)
for t in range(10):
    print('\nguess', t+1)
    flags = wordle.verify(guess)
    if wordle.solved:
        print(f'solved word "{guess}" in {t+1} steps')
        break

    inferer.update_priors(guess, flags)
    guess = inferer.next_guess(t)


guess 1
P A C E D 
- - - - - knoll in word bank: True
623 candidates left in iter | 

guess 2
B I G H T 
- - - - - knoll in word bank: True
86 candidates left in iter | 

guess 3
F L O U R 
- ! + - - knoll in word bank: True
knoll in loop!
7 candidates left in iter | 

guess 4
J O W L S 
- ! - + - knoll in word bank: True
0 candidates left in iter | 
['abode', 'acrid', 'addle', 'aegis', 'aggro', 'aired', 'alibi', 'aloes', 'ambit', 'amour', 'animo', 'antic', 'apron', 'argue', 'arses', 'aspen', 'attic', 'avert', 'axles', 'baits', 'banco', 'bares', 'basis', 'baulk', 'beats', 'begin', 'bench', 'betts', 'bijou', 'binge', 'bitty', 'blaze', 'bliss', 'blown', 'blurt', 'boggy', 'bombs', 'bonus', 'booth', 'bosky', 'bowel', 'brads', 'braze', 'brigg', 'brits', 'bruin', 'buffs', 'bulls', 'buoys', 'burro', 'butyl', 'cabot', 'caked', 'canes', 'cares', 'carve', 'caved', 'ceorl', 'chart', 'chewy', 'chirp', 'chubb', 'cirri', 'clare', 'click', 'clops', 'clyde', 'cogit', 'comas', 'conga', 'coppa', 'corms

ValueError: attempt to get argmax of an empty sequence

In [44]:
inferer.candidate_hist[1]

['picky', 'chirp', 'crypt', 'crimp']

In [11]:
print(len(printed_bank), len(inferer.word_bank))

1606 1605
